In [46]:
!pip3 install --quiet matplotlib numpy pandas scikit-learn seaborn

# Data Cleaning

In [47]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [48]:
# Changing the path to the root of the repository

if not os.path.exists('data'):
    current_path = os.getcwd()
    root_path = os.path.abspath(os.path.join(current_path, '..'))
    os.chdir(root_path)
    print(f'Changed working directory to: {os.getcwd()}')

In [49]:
# Load raw data
acorn_details = pd.read_csv('data/00_raw/acorn_details.csv', encoding='ISO-8859-1')
temperatures = pd.read_csv('data/00_raw/temperatures.csv', sep=';', decimal=',', encoding='utf-8')
uk_bank_holidays = pd.read_csv('data/00_raw/uk_bank_holidays.csv')
weather_daily = pd.read_csv('data/00_raw/weather_daily_darksky.csv')
weather_hourly = pd.read_csv('data/00_raw/weather_hourly_darksky.csv')

# Load processed data from parquet
group_4_daily_predict = pd.read_parquet('data/02_processed/parquet/group_4_daily_predict.parquet')
group_4_half_hourly_predict = pd.read_parquet('data/02_processed/parquet/group_4_half_hourly_predict.parquet')
group_4_daily = pd.read_parquet('data/02_processed/parquet/group_4_daily.parquet')
group_4_half_hourly = pd.read_parquet('data/02_processed/parquet/group_4_half_hourly.parquet')

In [50]:
# Fixing Datetime formats
temperatures['DateTime'] = pd.to_datetime(temperatures['DateTime'], format='mixed')
uk_bank_holidays['Bank holidays'] = pd.to_datetime(uk_bank_holidays['Bank holidays'], format='mixed')

# Convert all datetime-like columns in weather_daily to datetime format
datetime_columns = ['temperatureMaxTime', 'temperatureMinTime', 'apparentTemperatureMinTime', 
                   'apparentTemperatureHighTime', 'time', 'sunsetTime', 'sunriseTime', 
                   'temperatureHighTime', 'uvIndexTime', 'temperatureLowTime', 
                   'apparentTemperatureMaxTime', 'apparentTemperatureLowTime']

for col in datetime_columns:
    weather_daily[col] = pd.to_datetime(weather_daily[col])